## ___Missing Values___

Missing data, or Missing values, occur when __no data__ / __no value__ is stored for a certain observation within a variable. 

Missing data are a common occurrence both in data science competitions and in data in business settings, and can have a significant effect on the conclusions that can be drawn from the data. Incomplete data is an unavoidable problem in dealing with most data sources.

### ___Why is Data Missing?___

The source of missing data can be very different and here are just a few examples:

- A value is missing because it was forgotten or lost or not stored properly
- For a certain observation, the value of the variable does not exist
- The value can't be known or identified

Imagine for example that the data comes from a survey, and the data are entered manually into an online form. The data entry could easily forget to complete a field in the form, and therefore, that value for that form would be missing.

The person being asked may not want to disclose the answer to one of the questions, for example, their income. That would be then a missing value for that person.

Sometimes, a certain feature can't be calculated for a specific individual. For example, in the variable 'total debt as percentage of total income' if the person has no income, then the total percentage of 0 does not exist. Therefore it will be a missing value.

Together with understanding the source of missing data, it is important to understand the mechanisms by which missing fields are introduced in a dataset. Depending on the mechanism, we may choose to process the missing values differently. In addition, by knowing the source of missing data, we may choose to take action to control that source, and decrease the number of missing data looking forward during data collection.

### ___Missing Data Mechanisms___

There are 3 mechanisms that lead to missing data, 2 of them involve missing data randomly or almost-randomly, and the third one involves a systematic loss of data.

#### 1. ___Missing Completely at Random (MCAR)___

A variable is missing completely at random (MCAR) if the probability of being missing is the same for all the observations. 
When data is MCAR, there is absolutely no relationship between the data missing and any other values, observed or missing, within the dataset. In other words, those missing data points are a random subset of the data. There is nothing systematic going on that makes some data more likely to be missing than other.

If values for observations are missing completely at random, then disregarding those cases would not bias the inferences made.

#### 2. ___Missing at Random (MAR)___ 

MAR occurs when there is a systematic relationship between the propensity of missing values and the observed data. In other words, the probability an observation being missing depends only on available information (other variables in the dataset). For example, if men are more likely to disclose their weight than women, weight is MAR. The weight information will be missing at random for those men and women that decided not to disclose their weight, but as men are more prone to disclose it, there will be more missing values for women than for men.

In a situation like the above, if we decide to proceed with the variable with missing values (in this case weight), we might benefit from including gender to control the bias in weight for the missing observations.

#### 3. ___Missing Not at Random (MNAR)___

Missing of values is not at random (MNAR) if their being missing depends on information not recorded in the dataset. In other words, there is a mechanism or a reason why missing values are introduced in the dataset.

Examples:

MNAR would occur if people failed to fill in a depression survey because of their level of depression. Here, the missing of data is related to the outcome, depression.

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# to display the total number columns present in the dataset
pd.set_option('display.max_columns', None)

In [3]:
# let's load the titanic dataset

data = sns.load_dataset('titanic')
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


_In python, the missing values are stored as NaN, see for example the first row for the variable deck._

In [4]:
# you can determine the total number of missing values using
# the isnull method plus the sum method on the dataframe

data.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
# alternatively, you can call the mean method after isnull
# to visualise the percentage of the dataset that 
# contains missing values for each variable

data.isnull().mean()

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64

We can see that there are missing data in the variables Age, Deck (on which the passenger was travelling) and Embarked, which is the port from which the passenger got into the Titanic.

### ___Missing Data Not At Random (MNAR): Systematic Missing Values___

In this dataset, both the missing values of the variables Deck and Age, were introduced systematically. For many of the people who did not survive, the **age** they had or the **deck** they were staying in, could not be established. The people who survived could be asked for that information.

Can we infer this by looking at the data?

In a situation like this, we could expect a greater number of missing values for people who did not survive.

In [6]:
# we create a dummy variable that indicates whether the value
# of the variable deck is missing

data['deck_null'] = np.where(data.deck.isnull(), 1, 0)

# find percentage of null values
data.deck_null.mean()

0.7721661054994389

As expected, this value coincides with the one observed above when we called the .isnull().mean() method on the dataset.

In [8]:
# and then we evaluate the mean of the missing values in
# cabin for the people who survived vs the non-survivors.

# group data by Survived vs Non-Survived
# and find nulls for cabin
data.groupby(['survived'])['deck_null'].mean()

survived
0    0.877960
1    0.602339
Name: deck_null, dtype: float64

We observe that the percentage of missing values is higher for people who did not survive (0.87), respect to people that survived (0.60).
This finding is aligned with our hypothesis that the data is missing because after the people died, the information could not be retrieved.

Having said this, to truly underpin whether the data is missing not at random, we would need to get extremely familiar with the way data was collected. Analysing datasets, can only point us in the right direction or help us build assumptions.

In [10]:
# we repeat the exercise for the variable age:
# First we create a dummy variable that indicates
# whether the value of the variable Age is missing

data['age_null'] = np.where(data.age.isnull(), 1, 0)

# and then look at the mean in the different survival groups:
# there are more NaN for the people who did not survive
data.groupby(['survived'])['age_null'].mean()

survived
0    0.227687
1    0.152047
Name: age_null, dtype: float64

Again, we observe an increase in missing data for the people who did not survive the tragedy. The analysis therefore suggests: 

**There is a systematic loss of data: people who did not survive tend to have more information missing. Presumably, the method chosen to gather the information, contributes to the generation of these missing data.**

### ___Missing Data Completely At Random (MCAR)___

In the titanic dataset, there were also missing values for the variable Embarked, let's have a look.

In [11]:
# slice the dataframe to show only those observations with missing value for Embarked

data[data.embarked.isnull()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,deck_null,age_null
61,1,1,female,38.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True,0,0
829,1,1,female,62.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True,0,0


These 2 women were travelling together, Miss Icard was the maid of Mrs Stone.

A priori, there does not seem to be an indication that the missing information in the variable Embarked is depending on any other variable, and the fact that these women survived, means that they could have been asked for this information.

Very likely this missingness was generated at the time of building the dataset and therefore we could assume that it is completely random. We can assume that the probability of data being missing for these 2 women is the same as the probability for this variable to be missing for any other person. Of course this will be hard, if possible at all, to prove.

### ___Missing Data At Random (MAR)___

For this example, We will use the Lending Club loan book. we will look specifically at the variables employer name (emp_title) and years in employment (emp_length), declared by the borrowers at the time of applying for a loan. The former refers to the name of the company for which the borrower works, the second one to how many years the borrower has worked for named company.

Here we will show an example, in which a data point missing in one variable (emp_title) depends on the value entered on the other variable (emp_length).

In [0]:
# let's load the columns of interest from the Lending Club loan book dataset

data=pd.read_csv('/content/loan.csv', usecols=['emp_title','emp_length'])
data.head()

,emp_title,emp_length
0,NaN,10+ years
1,Ryder,< 1 year
2,NaN,10+ years
3,AIR RESOURCES BOARD,10+ years
4,University Medical Group,1 year


In [0]:
# let's check the amount of missing data
data.isnull().mean()

emp_title     0.057988
emp_length    0.000000
dtype: float64

Around 6% of the observations contain missing data for employment title.

In [0]:
# let's peek at the different employer names

print('Number of different employer names: {}'.format(len(data.emp_title.unique())))
data.emp_title.unique()[0:20]

Number of different employer names: 299273


array([nan, 'Ryder', 'AIR RESOURCES BOARD', 'University Medical Group',
       'Veolia Transportaton', 'Southern Star Photography',
       'MKC Accounting ', 'Starbucks', 'Southwest Rural metro', 'UCLA',
       'Va. Dept of Conservation/Recreation', 'Target', 'SFMTA',
       'Internal revenue Service', "Chin's Restaurant", 'Duracell',
       'Connection Inspection', 'Network Interpreting Service',
       'Archdiocese of Galveston Houston', 'Osram Sylvania'], dtype=object)

In [0]:
# let's inspect the variable emp_length
data.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a'], dtype=object)

In [0]:
# let's look at the percentage of borrowers within
# each label / category of the emp_length variable

data.emp_length.value_counts() / len(data)

10+ years    0.328573
2 years      0.088880
< 1 year     0.079566
3 years      0.078913
1 year       0.064341
5 years      0.062774
4 years      0.059196
n/a          0.050514
7 years      0.050254
8 years      0.049534
6 years      0.048401
9 years      0.039055
Name: emp_length, dtype: float64

The label 'n/a', which in virtue could be similar to NaN, represents also around 5% of the dataset. So there could be a relationship between missing values in emp_title and 'n/a' in emp_length. Let's have a look.

In [0]:
# the variable emp_length has many categories. I will summarise it
# into 3 for simplicity:'0-10 years' or '10+ years' or 'n/a'

# let's build a dictionary to re-map emp_length to just 3 categories:

length_dict = {k:'0-10 years' for k in data.emp_length.unique()}
length_dict['10+ years']='10+ years'
length_dict['n/a']='n/a'

# let's look at the dictionary
length_dict

{'1 year': '0-10 years',
 '10+ years': '10+ years',
 '2 years': '0-10 years',
 '3 years': '0-10 years',
 '4 years': '0-10 years',
 '5 years': '0-10 years',
 '6 years': '0-10 years',
 '7 years': '0-10 years',
 '8 years': '0-10 years',
 '9 years': '0-10 years',
 '< 1 year': '0-10 years',
 'n/a': 'n/a'}

In [0]:
# let's re-map the emp_length

data['emp_length_redefined'] = data.emp_length.map(length_dict)
data.emp_length_redefined.unique()

array(['10+ years', '0-10 years', 'n/a'], dtype=object)

In [0]:
# let's calculate the proportion of working years
# with same employer for those who miss data on employer name

# number of borrowers for whom employer name is missing
value = len(data[data.emp_title.isnull()])

# % of borrowers for whom employer name is missing 
# within each category of employment length
data[data.emp_title.isnull()].groupby(['emp_length_redefined'])['emp_length'].count().sort_values() / value

emp_length_redefined
10+ years     0.054006
0-10 years    0.082457
n/a           0.863537
Name: emp_length, dtype: float64

The majority of the missing values in the job title (emp_title) supplied by the borrower coincides with the label n/a of employment length. This supports the idea that the 2 variables are related.

In [0]:
# let's do the same for those bororwers who reported
# the employer name

# number of borrowers for whom employer name is present
value = len(data.dropna(subset=['emp_title']))

# % of borrowers within each category
data.dropna(subset=['emp_title']).groupby(['emp_length_redefined'])['emp_length'].count().sort_values() / value

emp_length_redefined
n/a           0.000467
10+ years     0.345475
0-10 years    0.654059
Name: emp_length, dtype: float64

The number of borrowers who have reported an employer name and indicate n/a as employment length are minimal. Further supporting the idea that the missing values in employment length and employment length are related.

'n/a' in 'employment length' could be supplied by people who are retired, or students, or self-employed. In all of those cases there would not be a number of years at employment to provide, therefore the customer would enter 'n/a' and leave empty the form at the side of 'employer_name'.

In a scenario like this, a missing value in the variable emp_title depends on or is related to the 'n/a' label in the variable emp_length. And, this missing value nature is, in principle, independent of the variable we want to predict (in this case whether the borrower will repay their loan). How this will affect the predictions is unknown.